In [1]:
import numpy as np
import math

In [90]:
value_upper=1e-2
dim=5
pmin = np.random.randint(1,10,dim)
pmax = pmin + np.random.randint(1,10,dim)
B=1

In [91]:
pmin,pmax

(array([9, 7, 5, 1, 8]), array([10, 13, 13,  5, 15]))

In [92]:
def psi(z, p_min, p_max):
    psi = ((p_max * np.exp(1) / p_min) ** z) * (p_min / np.exp(1))
    return psi.reshape(-1)

In [104]:
def generate_multi_dimensional(p_min,p_max,value_upper_bound=1e-3,B=1,n=5):
    max_len = round(B/value_upper_bound*2* n * 1.25 ) # decide the maximum length of this data flow
    value = np.array([])
    weight = np.zeros([n,max_len])
    for i in range(max_len):
        val = np.random.uniform(0,value_upper_bound)
        value = np.append(value, [val])
        weight[:,i] = np.random.uniform(val/pmax, val/pmin)
    return (weight, value)

In [105]:
(weight, value) = generate_multi_dimensional(pmin,pmax,value_upper,B,dim)

In [106]:
np.mean(1/(weight/value.reshape(1,-1)), axis=1)

array([ 9.48314332,  9.34750926,  7.72951405,  2.01646198, 10.74383753])

In [107]:
def multi_online_knapsack(weights, values, B,dim):
    '''
    para: weights [dim,len]
    para: values [len]
    return: 
    condcucts multi-dimensional online knapsack problem
    '''
    z = np.zeros(dim) # the current fraction
    ps = 1/(weights/values.reshape(1,-1))
    p_min = np.min(ps, axis=1)
    p_max = np.max(ps, axis=1)# pmin and pmax of this flow
    cur_val = np.zeros(dim)# a list, each entry i representing the current accumulated value of the ith bin
    cur_weight = np.zeros(dim)# a list, each entry i representing the current occupied weight of the ith bin
    B = B*np.ones(dim)
    for j in range(weights.shape[1]):
        non_overfill = (cur_weight+weights[:,j])<=B
        # if at least one not overfilled
        if np.sum(non_overfill)>0:
            psi_z = psi(z, p_min, p_max)
            # rule out those overfilled
            dst = (values[j]/(weights[:,j])).reshape(-1) * (np.asarray(non_overfill, dtype=int).reshape(-1)) # density
            satis_cond = np.asarray(dst > psi_z, dtype=int)
            argmax = np.argmax(dst * satis_cond)# find out the max density index
            # update
            cur_weight[argmax] += weights[argmax,j]
            z[argmax] = z[argmax] + (weights[argmax,j] / B[argmax])
            cur_val[argmax] += values[j]
    return (cur_val,p_min,p_max, cur_weight)

In [108]:
multi_online_knapsack(weight, value, 1, dim)

(array([0.95953178, 1.28768872, 0.78841597, 0.        , 3.15298266]),
 array([9.0011787 , 7.00089215, 5.00394668, 1.0000871 , 8.00334645]),
 array([ 9.99906379, 12.99322847, 12.98855342,  4.99689682, 14.99891331]),
 array([0.10031604, 0.1129633 , 0.07025705, 0.        , 0.26246941]))

In [45]:
weight.shape[1]

20